In [4]:
from tqdm import tqdm
import psycopg
import os
import json
from dotenv import load_dotenv


load_dotenv()

True

# Update DB with the new tables

In [7]:
conn = psycopg.connect(
    dbname="youtubestay",
    user="postgres",
    password=os.getenv("POSTGRE_PASSWORD"),
    host="localhost",
    port="5432"
)


cur = conn.cursor()

cur.execute("""
    CREATE TABLE entites_spatiales (
        id_entite_spatiale TEXT PRIMARY KEY,
        label TEXT NOT NULL,
        latitude FLOAT NOT NULL,
        longitude FLOAT NOT NULL 
    )
""")

cur.execute("""
    CREATE TABLE entites_spatiales_videos (
        id_entite_spatiale TEXT REFERENCES entites_spatiales(id_entite_spatiale) ON DELETE CASCADE,
        id_video TEXT REFERENCES videos(id_video) ON DELETE CASCADE,
        PRIMARY KEY (id_video, id_entite_spatiale)
    )
""")

cur.execute("""
    CREATE TABLE entites_spatiales_chaines (
        id_entite_spatiale TEXT REFERENCES entites_spatiales(id_entite_spatiale) ON DELETE CASCADE,
        id_chaine TEXT REFERENCES chaines(id_chaine) ON DELETE CASCADE,
        PRIMARY KEY (id_chaine, id_entite_spatiale)
    )
""")


conn.commit()
cur.close()
conn.close()


# Fill the spacial_entities_videos table

In [5]:
def getContext(title,description,tags):
    videoContext = ''
    videoContext+=title
    videoContext+= '\n'+description
    videoContext += '\n'+ ', '.join(tags)
    return videoContext

- Test

In [6]:
title = "Autonomie alimentaire.  Être autosuffisant sur petite surface !"
description = """
Découvrez le Pirate de la Permaculture et son autonomie alimentaire sur toute petite surface. Picro arrive à être autosuffisant sur une surface d'à peine 800m2... de quoi rêver.

Adhérez à cette chaîne pour obtenir des avantages :
https://www.youtube.com/channel/UC9Q8WeyCb3yxySC3P3mGpBw/join
Pour me soutenir, suivez ce lien : https://fr.tipeee.com/le-jardin-d-emerveille

Au sommaire :
0:00 - Présentations
0:53 - Quelles productions
4:24 - Comment calculer son autosuffisance !
5:00 - Réduire sa dépendance énergétique
6:20 - Visite du lieu
8:08 - Surface et organisation de la production
10:58 - Gestion de l'eau
11:51 - Jardin forêt ?
13:24 - Le pirate Picro et sa chaîne YouTube.

La chaîne YouTube de Picro : https://www.youtube.com/user/piiicro

Pour me soutenir, suivez ce lien : https://fr.tipeee.com/le-jardin-d-emerveille

Merci à vous tous les permapotes d'avoir regardé cette vidéo. :)
Cliquez sur ce lien pour vous abonner : 
https://www.youtube.com/channel/UC9Q8WeyCb3yxySC3P3mGpBw
"""

tags = ["Autonomie alimentaire. Être autosuffisant sur petite surface !","permaculture","plantes","jardin","biodiversité","agroécologie","potager","des merveilles","Autonomie alimentaire","Produire sa nourriture","comment créer son potager bio","comment démarrer son potager","comment démarrer un potager","comment faire un potager bio","comment préparer son potager","créer son jardin","créer son potager","faire un potager","etre autosuffisant","Autosuffisant sur petite surface"]

videoTestContexte = getContext(title, description, tags)

print(videoTestContexte)


Autonomie alimentaire.  Être autosuffisant sur petite surface !

Découvrez le Pirate de la Permaculture et son autonomie alimentaire sur toute petite surface. Picro arrive à être autosuffisant sur une surface d'à peine 800m2... de quoi rêver.

Adhérez à cette chaîne pour obtenir des avantages :
https://www.youtube.com/channel/UC9Q8WeyCb3yxySC3P3mGpBw/join
Pour me soutenir, suivez ce lien : https://fr.tipeee.com/le-jardin-d-emerveille

Au sommaire :
0:00 - Présentations
0:53 - Quelles productions
4:24 - Comment calculer son autosuffisance !
5:00 - Réduire sa dépendance énergétique
6:20 - Visite du lieu
8:08 - Surface et organisation de la production
10:58 - Gestion de l'eau
11:51 - Jardin forêt ?
13:24 - Le pirate Picro et sa chaîne YouTube.

La chaîne YouTube de Picro : https://www.youtube.com/user/piiicro

Pour me soutenir, suivez ce lien : https://fr.tipeee.com/le-jardin-d-emerveille

Merci à vous tous les permapotes d'avoir regardé cette vidéo. :)
Cliquez sur ce lien pour vous abonn

In [8]:
from langchain_ollama import ChatOllama
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

llm = ChatOllama(model="llama3.2:3b")

system_template = """
Tu es un extracteur d'entités de géolocalisation en France. 
Tu reçois un texte et tu dois retourner **uniquement** la liste des entités géographiques françaises présentes, 
comme les villes, communes, villages, quartiers, etc.

Formate ta réponse comme une liste Python : ["entité1", "entité2", ...]
Tout doit être en **minuscules** et sans doublons. Aucune explication.
"""

user_template = "Contexte : {contexte}"

system_message = SystemMessagePromptTemplate.from_template(system_template)
user_message = HumanMessagePromptTemplate.from_template(user_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message, user_message])


In [10]:
def getSpacialEntities(context):
    messages = chat_prompt.format_messages(contexte=context)
    response = llm.invoke(messages)
    
    try:
        entities = eval(response.content.strip())
        if isinstance(entities, list):
            return [e.lower().strip() for e in entities if isinstance(e, str)]
    except:
        pass
    return []

In [12]:
# Exemple de texte avec des noms de lieux
texte_contenu = """
Lors de mon voyage en Provence, j’ai visité Marseille, le quartier du Panier, Aix-en-Provence 
et un petit village appelé Eygalières. Ensuite, nous sommes allés à Nice et dans le Vieux-Nice.
"""
getSpacialEntities(texte_contenu)

['marseille', 'aix-en-provence', 'eygalières', 'nice', 'vieux-nice']